In [1]:
import numpy as np
import pandas as pd

In [2]:
transactions = pd.read_csv('data-files/trx_data.csv')

In [5]:
transactions.info()
transactions.head(10)# 1번 사용자가 2번, 68번 제품을 2번 샀다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62483 entries, 0 to 62482
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerId  62483 non-null  int64 
 1   products    62483 non-null  object
dtypes: int64(1), object(1)
memory usage: 976.4+ KB


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2
5,6,144|144|55|266
6,7,135|206|259
7,8,79|8|8|48
8,9,102|2|2|297
9,10,84|77|290|260


In [6]:
products = transactions['products'].str.split('|') # |를 각각 개별적인 요소로 자르기
products

0                                             [20]
1        [2, 2, 23, 68, 68, 111, 29, 86, 107, 152]
2               [111, 107, 29, 11, 11, 11, 33, 23]
3                                       [164, 227]
4                                           [2, 2]
                           ...                    
62478                       [103, 103, 8, 48, 126]
62479    [124, 37, 37, 78, 124, 124, 37, 8, 35, 8]
62480                                         [24]
62481                                        [167]
62482                                        [150]
Name: products, Length: 62483, dtype: object

In [8]:
transactions['products'] = products.map(lambda x: [int(p) for p in x]) # 문자열 요소들을 숫자열로 바꿈

In [9]:
transactions.head()

,customerId,products
0,0,[20]
1,1,"[2, 2, 23, 68, 68, 111, 29, 86, 107, 152]"
2,2,"[111, 107, 29, 11, 11, 11, 33, 23]"
3,3,"[164, 227]"
4,5,"[2, 2]"


In [11]:
print(transactions.iloc[1, 1][0])
transactions.head()

2


,customerId,products
0,0,[20]
1,1,"[2, 2, 23, 68, 68, 111, 29, 86, 107, 152]"
2,2,"[111, 107, 29, 11, 11, 11, 33, 23]"
3,3,"[164, 227]"
4,5,"[2, 2]"


In [19]:
data_list = []
for idx, transaction in transactions.iterrows(): # iterrows: 행의 반복
    for product in transaction["products"]: # list를 합치기
        data_list.append({"customerId":transaction['customerId'], "productId":product})

In [21]:
data_list[:5] # 0번 사용자는 20번의 제품을 가지고 있고, ...

[{'customerId': 0, 'productId': 20},
 {'customerId': 1, 'productId': 2},
 {'customerId': 1, 'productId': 2},
 {'customerId': 1, 'productId': 23},
 {'customerId': 1, 'productId': 68}]

In [22]:
data = pd.DataFrame(data_list)

In [23]:
data.head() # 거래한 내역 행으로 처리

,customerId,productId
0,0,20
1,1,2
2,1,2
3,1,23
4,1,68


In [24]:
data['purchase_count'] = np.ones(len(data))

In [25]:
data.head() # 처음에는 다 1

,customerId,productId,purchase_count
0,0,20,1.0
1,1,2,1.0
2,1,2,1.0
3,1,23,1.0
4,1,68,1.0


In [35]:
data2 = data.groupby(['customerId', 'productId'], as_index=False).count() # 인덱스 구성 X

In [36]:
data2.head()
# data2.columns = ['userid', 'itemid', 'rating']

,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


In [37]:
data2['purchase_count'].describe() # 선호도에 대한 정보

count    133585.000000
mean          1.583097
std           1.116338
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max          75.000000
Name: purchase_count, dtype: float64

In [38]:
from surprise.reader import Reader
from surprise import Dataset

In [40]:
reader=Reader()
ds = Dataset.load_from_df(data2, reader) # dataframe으로부터 읽어와라! reader객체를 가져오라고 하는 것

In [42]:
ds.raw_ratings # 맨 끝은 id 

[(0, 1, 2.0, None),
 (0, 13, 1.0, None),
 (0, 19, 3.0, None),
 (0, 20, 1.0, None),
 (0, 31, 2.0, None),
 (0, 52, 1.0, None),
 (0, 69, 2.0, None),
 (0, 93, 3.0, None),
 (0, 136, 2.0, None),
 (0, 157, 1.0, None),
 (0, 198, 1.0, None),
 (0, 216, 1.0, None),
 (0, 255, 2.0, None),
 (0, 256, 1.0, None),
 (0, 260, 5.0, None),
 (1, 2, 6.0, None),
 (1, 23, 1.0, None),
 (1, 25, 1.0, None),
 (1, 29, 1.0, None),
 (1, 61, 2.0, None),
 (1, 68, 2.0, None),
 (1, 86, 3.0, None),
 (1, 107, 1.0, None),
 (1, 111, 1.0, None),
 (1, 122, 1.0, None),
 (1, 135, 1.0, None),
 (1, 146, 1.0, None),
 (1, 151, 3.0, None),
 (1, 152, 3.0, None),
 (1, 216, 1.0, None),
 (1, 227, 1.0, None),
 (1, 241, 1.0, None),
 (1, 274, 1.0, None),
 (1, 285, 1.0, None),
 (1, 293, 1.0, None),
 (1, 296, 1.0, None),
 (2, 11, 3.0, None),
 (2, 23, 1.0, None),
 (2, 29, 1.0, None),
 (2, 33, 1.0, None),
 (2, 107, 1.0, None),
 (2, 111, 1.0, None),
 (3, 25, 1.0, None),
 (3, 55, 1.0, None),
 (3, 100, 1.0, None),
 (3, 121, 1.0, None),
 (3, 164, 1